In [1]:
!mkdir sieve
# make dir sieve

In [4]:
!cd sieve
# go to that dir

In [36]:
def Eratosthenes(upper_bound):
    # Return a list of the primes BELOW the upper_bound
    # first, create a list with all primes set to `True`
    iteration = 1
    prime = [True] * upper_bound 
    # an array of length upper bound
    for p in range(3, upper_bound, 2):
        # the range is skipped by intervals of 2// faster than skipping in units of 1.. since 2 and its multiples are ruled out anyway
        if p > (upper_bound**.5):
            # implementation explained below
            break
        if prime[p]==True: 
            # encountered a index number that is still true [as one is counting from 3 upwards] i.e. prime
            # remember that all multiples of p are not prime, whether or not p is prime [but the way this is written... the 'whether or not p is prime' case is not encountered]
            for i in range(p * p, upper_bound, 2 * p): # we skip in terms of 2*p because.. rule out the multiples of 2
                prime[i] = False
                iteration = iteration + 1
    #iteration = 0
    # don't forget about the number "2"
    return [2] + [p for p in range(3, upper_bound, 2) if prime[p]]
    
# n =a*b if n is not prime.. then atleast one of a,b must be less than the square root of n [food for thought: use proof by contradiciton for the statement]

In [37]:
%%writefile prime_sieve.py
def Eratosthenes(upper_bound):
    # Return a list of the primes below the upper_bound
    # first, create a list with all values set to true
    prime = [True] * upper_bound
    for p in range(3, upper_bound, 2):
        if p > (upper_bound**.5):
            break
        if prime[p]:
           
            for i in range(p * p, upper_bound, 2 * p):
                prime[i] = False
    
    return [2] + [p for p in range(3, upper_bound, 2) if prime[p]]

# same as the above.. written to file..

Writing prime_sieve.py


In [ ]:
# check out https://github.com/IBM-Cloud/get-started-python
# sometime... for more on CLI CF.. etc..

# the aim to to develop the app entirely through DSX notebooks..


In [38]:
%%writefile my_flask_app.py
from flask import Flask, Response, jsonify
from flask_restplus import Api, Resource, fields, reqparse
from flask_cors import CORS, cross_origin
import os

# the app itsel
app = Flask(__name__)
CORS(app)
api = Api(app, version='1.0', title='APIs for Python Functions', validate=False)

# create namespace to organize the api and docs
ns = api.namespace('primality', 'Returns a list of all primes below a given upper bound')

# load the algorithm above
# from the said file.. import the said class as algo..
from prime_sieve import Eratosthenes as algo

# model the input data
model_input = api.model('Enter the upper bound:', {
    "UPPER_BOUND": fields.Integer(maximum=10e16)})

# the input data type here is Integer. You can change this to whatever works for your app.

# On Bluemix, get the port number from the environment variable PORT
# When running this app on the local machine, default the port to 8080

port = int(os.getenv('PORT', 8080))


# The ENDPOINT
@ns.route('/sieve') # the endpoint
class SIEVE(Resource):
    @api.response(200, "Success", model_input) # return a formatted response
    @api.expect(model_input) # expcect the required the input data
    def post(self): # prefer POST
        parser = reqparse.RequestParser() # parse the args
        parser.add_argument('UPPER_BOUND', type=int) # get the data
        args = parser.parse_args()
        inp = int(args["UPPER_BOUND"]) # our input data
        result = algo(inp) # apply algo
        return jsonify({"primes": result})
    
        # that sure is a lot of steps :D
# run      
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=port, debug=True)

Writing my_flask_app.py


### Supporting documents ...  lets see

    include some other files in order for our api to behave properly when we try to deploy the app to Bluemix. These files are:

    manifest.yml, this includes basic information about your app, such as the name and memory
    setup.py
    README.md
    requirements.txt
    Procfile

In [39]:
%%writefile manifest.yml
---
applications:
 - name: PRIMALITY_UNIQUE # name your app whatever you'd like
   random-route: true
   memory: 256M # can adjust this based on your service 

Writing manifest.yml


In [40]:
%%writefile Procfile
web: python my_flask_app.py

Writing Procfile


In [41]:
%%writefile README.md
"Getting Started with Python Algos on Bluemix"

Overwriting README.md


In [42]:
%%writefile requirements.txt
Flask==0.11.1
cloudant==2.4.0
flasgger==0.6.4
Flask-Cors==3.0.2
Flask-RESTful==0.3.6
flask-restplus==0.9.2
gevent==1.2.1

Writing requirements.txt


In [43]:
%%writefile setup.py
"""
Hello World app for deploying Python functions as APIs on Bluemix
"""

# Always prefer setuptools over distutils
from setuptools import setup, find_packages
# To use a consistent encoding
from codecs import open
from os import path

here = path.abspath(path.dirname(__file__))

# Get the long description from the README file
with open(path.join(here, 'README.md'), encoding='utf-8') as f:
    long_description = f.read()

setup(
    name='primality_test',
    version='1.0.0',
    description='Running Python apps on Bluemix',
    long_description=long_description,
    url='https://github.com/IBM-Bluemix/python-hello-world-flask',
    license='Apache-2.0'
)

Writing setup.py


In [44]:
%%writefile LICENSE
"""Apache License
                           Version 2.0, January 2004
                        http://www.apache.org/licenses/

   TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION ..."""

Writing LICENSE


### Writing to bluemix...